## FUAM Migration Notebook 

Since the first published version, we got numerous feedbacks from you.

We were listening and we find the right, mature structure for files, activity ids and silver tables.

We try to avoid in the future such a migration notebooks in the future releases of FUAM.

In [ ]:
import pyspark.sql.functions as f

### Change activity ids to upper case
In the new version specific IDs in the activity table have been put to Upper Case to be able to join to other tables in Power BI. This script updates these IDs one time

In [ ]:
df_activities = spark.sql("SELECT * FROM FUAM_Lakehouse.activities")
for co in df_activities.columns:
    if co in ['ActivityId','ArtifactId','CapacityId','DashboardId','DataflowId','DatasetId','DatasourceId','FolderObjectId','GatewayId','Id','ItemId','ReportId','UserId','WorkspaceId','','','','','','','','','',]:
        df_activities = df_activities.withColumn(co, f.upper(df_activities[co]))
df_activities.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable("activities")

### Delete silver tables
Old silver tables will be deleted, since they are now available in a seperate Lakehouse

In [ ]:
tables_to_delete = ['capacities_silver','active_items_silver','tenant_settings_silver','delegated_tenant_settings_overrides_silver','workspaces_silver','capacity_metrics_by_item_kind_by_day_silver','capacity_metrics_by_timepoint_silver','capacity_metrics_by_item_by_operation_by_day_silver']

for table in spark.catalog.listTables():
    table_name = table[0]
    if table_name in tables_to_delete:
        print(table_name + ' will be deleted')
        spark.sql("DROP TABLE "+ table_name)

### Move old files into new file structure
Existing folders in the lakehouse will be moved to the history folder to enable better readability. 
This will just put into an "old" folder. After updating FUAM, the new pipeline will create a "raw" folder, where the daily new data will be persisted.

In [ ]:
folders_to_move = ['active_items','activities','capacities','delegated_tenant_settings_overrides','inventory','refreshables','tenant_settings','workspaces']

for folder in mssparkutils.fs.ls("Files"):
    if folder.name in folders_to_move:
        print(folder.name + ' will be moved')
        src_path = folder.path
        dest_path = src_path.replace('Files/' + folder.name , 'Files/history/' + '/old/' +  folder.name)
        print(dest_path)
        mssparkutils.fs.mv(src_path, dest_path, True)